In [1]:
import sys
sys.path.append('../src')
import exlib

In [3]:
import torch
import torch.nn.functional as F
from exlib.datasets.massmaps import MassMaps, MassMapsConvnet

batch_size = 5
DATA_DIR = "~/foo/data/cosmogrid"

test_dataset = MassMaps(DATA_DIR, 
                        split='test')
loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)
model = MassMapsConvnet(DATA_DIR)

FileNotFoundError: [Errno 2] No such file or directory: '~/foo/data/cosmogrid/mass_maps/mass_maps_test/test_x.npy'

In [4]:
for X,y in loader:
    X,y = X.float(), y.float()
    out = model(X)
    loss = F.mse_loss(out, y)
    print(out, y, loss)
    break

NameError: name 'loader' is not defined

In [5]:
from exlib.explainers.torch_explainer import *
from exlib.explainers.common import patch_segmenter
from exlib.evaluators.attributions import NNZ


eik = {
    "segmentation_fn": patch_segmenter,
    "top_labels": 5, 
    "hide_color": 0, 
    "num_samples": 1000
}
gimk = {
    "positive_only": False
}
explainer = LimeImageClass(model, 
                      explain_instance_kwargs=eik, 
                      get_image_and_mask_kwargs=gimk)
evaluator = NNZ()
expln = explainer(X)
evaluator(X,expln.attributions, normalize=True)

NameError: name 'model' is not defined

In [ ]:
from skimage.segmentation import mark_boundaries
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

explanation = expln.explainer_output[0]

temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=8, hide_rest=False)
X_min, X_max = temp.min(), temp.max()
img_boundry1 = mark_boundaries((temp-X_min)/(X_max-X_min), mask)
plt.imshow(img_boundry1, vmin=0, vmax=1, cmap='bwr')

In [ ]:
explainer = TorchImageSHAP(model)
expln = explainer(X)
evaluator(X,expln.attributions, normalize=True)

In [6]:
y = torch.rand(4,3,32,32)

In [8]:
y.argmax(dim=1, keepdim=True).shape

torch.Size([4, 1, 32, 32])